In [12]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library(stringr) 

In [13]:
doubles <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_doublesSameGroup.rds")
doubles <- doubles[doubles$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
doubles[,c("std.error", "statistic", "p.value")] = NULL
doubles = reshape(doubles, idvar = "respGene", timevar = "term",  direction = "wide")
rownames(doubles) = doubles$respGene
doubles$respGene = NULL
colnames(doubles) = sapply( colnames(doubles), function(x){strsplit(x,"estimate.")[[1]][2]})


In [14]:
head(doubles)

,K_0,K_1,K_2,K_3,K_4,K_5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AI314180,0.011444335,0.076045764,0.051836802,0.06058460,-0.06955889,-0.17319416
Abcc1,-0.016205187,-0.017736019,0.018213094,-0.01751918,0.21419332,-0.27525200
Acod1,0.040686038,-0.078986749,-0.113114151,0.16680908,-0.03095431,0.15079135
Akr1a1,0.006097903,-0.070728779,0.040799512,-0.02336757,-0.20688385,-0.02894113
Alas1,0.006392998,0.002413812,-0.006475581,-0.06575484,0.01197551,-0.03349484
Alox5ap,-0.006786062,-0.115397121,0.115904367,0.02666414,0.01756842,0.09002909


In [15]:
combos <- readRDS("./../Notebooks/CombinatorialPerturbations/RDSFiles/ComboEffects_lm_residuals.rds")
combos <- combos[combos$term %ni% c("(Intercept)", "n_genes", "mt_frac", paste0("leiden", 1:9)),]
combos[,c("std.error", "statistic", "p.value")] = NULL
combos = reshape(combos, idvar = "respGene", timevar = "term",  direction = "wide")
rownames(combos) = combos$respGene
combos$respGene = NULL
colnames(combos) = sapply( colnames(combos), function(x){strsplit(x,"estimate.")[[1]][2]})

In [16]:
head(combos)

,K_0,K_0WK_1,K_0WK_2,K_0WK_3,K_0WK_4,K_0WK_5,K_1,K_1WK_2,K_1WK_3,K_1WK_4,⋯,K_2,K_2WK_3,K_2WK_4,K_2WK_5,K_3,K_3WK_4,K_3WK_5,K_4,K_4WK_5,K_5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AI314180,0.0177419902,0.023616034,0.015147990,0.01853934,-0.04283023,0.08807579,0.003134237,0.11911014,0.067868539,0.001933236,⋯,0.025505984,0.039911636,-0.05816264,-0.0004189604,0.004775926,-0.01087277,0.03374760,0.0354271341,-0.02218737,-0.04202507
Abcc1,-0.0019303524,-0.010667928,-0.037595252,0.02513075,0.10844025,-0.06708029,-0.006876538,0.05157873,0.030669683,0.098433436,⋯,0.001977992,0.027703008,0.16145588,0.0396236619,0.018012907,-0.03530264,-0.09379990,-0.0004456438,0.33206340,-0.04169053
Acod1,0.0316256224,0.006961726,0.021547024,0.05206471,0.07651565,0.10174531,-0.036595821,-0.02258265,-0.008615383,0.060569299,⋯,-0.032005309,0.032330391,-0.01564057,0.1016252383,0.039036084,0.03466344,0.11574894,0.0639403980,0.12370651,0.07742290
Akr1a1,0.0006242596,-0.065574081,-0.013412927,-0.05530001,0.06266195,-0.05966461,-0.016953725,0.02212735,-0.048007086,-0.018679508,⋯,0.065520337,0.034269443,-0.01117202,0.0412736287,-0.032993804,-0.02034791,-0.06475619,0.0310538266,-0.04493234,0.04029752
Alas1,0.0194794837,-0.045359707,0.035774566,0.01938750,0.04701501,-0.15580484,-0.027958642,0.04067934,-0.062844266,-0.054685979,⋯,0.029076667,0.004379683,0.12857489,0.1059764073,-0.030150307,0.01632723,-0.03614840,0.0342867249,0.27329661,0.02810747
Alox5ap,0.0060830991,0.016090787,-0.001326297,0.04843851,-0.01089988,-0.02222179,-0.030469223,-0.03034116,0.039370152,-0.147942325,⋯,0.018033909,-0.017412220,-0.03431968,-0.0229946651,0.048023057,-0.01122091,-0.19628440,-0.0031723719,0.04144396,-0.01215041


In [17]:
allPred = data.frame()

for( i in c("K_0", "K_1", "K_2", "K_3", "K_4", "K_5")){       
    myR2 = round(computeR2(response = doubles[,i], prediction = 2*combos[,i] ), digits=2)
   
    
    myTemp =  data.frame("Predictedvalues" = 2*combos[,i],
                         "Observedvalues" = doubles[,i],
                         "Comb"=i,
                         "Gene" = rownames(combos),
                         "R2" = myR2,
                         "corS" = round(cor(doubles[,i], combos[,i], method="spearman"), digits=2),
                         "pearsonS" = round(cor(doubles[,i], combos[,i], method="pearson"), digits=2)
                        )
     v = prcomp(myTemp[,c("Predictedvalues", "Observedvalues")])$rotation
     myTemp$PcaFirstCompSlope = round(v[2,1]/v[1,1], digits = 2)
    
    allPred = rbind(allPred, myTemp)
      
}


In [18]:
head(allPred)

,Predictedvalues,Observedvalues,Comb,Gene,R2,corS,pearsonS,PcaFirstCompSlope
,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.035483980,0.011444335,K_0,AI314180,0.54,0.74,0.75,1.26
2,-0.003860705,-0.016205187,K_0,Abcc1,0.54,0.74,0.75,1.26
3,0.063251245,0.040686038,K_0,Acod1,0.54,0.74,0.75,1.26
4,0.001248519,0.006097903,K_0,Akr1a1,0.54,0.74,0.75,1.26
5,0.038958967,0.006392998,K_0,Alas1,0.54,0.74,0.75,1.26
6,0.012166198,-0.006786062,K_0,Alox5ap,0.54,0.74,0.75,1.26


In [19]:
allPred <- allPred[allPred$Comb != "K_5",]

In [20]:
allPred$Comb <- sapply(allPred$Comb, function(x){str_replace(x,"_","")})

In [21]:
allPred[allPred$R2 <0, "R2"] = 0

In [23]:
options(repr.plot.width=15, repr.plot.height=4)
pdf("Figure_5C.pdf", width = 15, height = 4)

ggplot(allPred, aes(x=Predictedvalues, y=Observedvalues)) + 
        geom_point(alpha=0.5, shape=15, color="blue") + 
        facet_wrap( ~ Comb, ncol=6)+
        geom_text(aes(label= paste0("s = ",PcaFirstCompSlope), color="red"), x=-0.25, y=0.75, size=6)+
        geom_text(aes(label= paste0("R^2 = ",R2), color="red"), x=0, y=-1, size=6)+
        geom_abline(linetype="dashed")+
        geom_abline(aes(slope = PcaFirstCompSlope,intercept=0,color="red"))+
        theme_minimal()+
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 15),
              strip.text = element_text(size = 15),
              legend.position='none')+
        xlim(-1,1)+ylim(-1,1)+
        ylab("Observed fold change in \n double KO cells (lg2)")+
        xlab("Sum of observed fold changes in single KO cells (lg2)")

dev.off()

png 
  2